# A brief experiment to investigate the sources of random wiring performance benefits/deficits in the CNN-LSTM model
See Section 6c of the manuscript

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/USC Random NN')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Some xarray functions are currently broken with version 5.0.0
!pip install -I importlib-metadata==4.13.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.1.2 requires typing-extensions<4.2.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.3.0 which is incompatible.
spacy 3.4.1 requires typing-extensions<4.2.0,>=3.7.4; python_version < "3.8", but you have typing-extensions 4.3.0 which is incompatible.
confection 0.0.2 requires typing-extensions<4.2.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.3.0 which is incompatible.


In [ ]:
!pip install eofs --quiet
import numpy as np
import xarray as xr
import pandas as pd
from utils import *
from utils_cnn_lstm import *
results_path = results_path + 'weight_freezing/cnn_lstm_rand_dense/'

from random_nn import *
import keras
from keras import Sequential, Model
from keras.layers import *
from keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.utils import plot_model
tf.get_logger().setLevel('ERROR')
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
from IPython.display import display

from scipy import stats

tf.keras.utils.set_random_seed(21)

In [ ]:
vars_to_predict = ['tas', 'diurnal_temperature_range', 'pr', 'pr90']
simus = ['ssp126',
         'ssp370',
         'ssp585',
         'hist-GHG',
         'hist-aer']
slider = 10  # sliding time window

# Selects all of 2081-2100 data as validation
#val_idx = list(range(75,86)) + list(range(161,172)) + list(range(403,414))

# Selects first two years of every decade from 2050 onward as validation
'''
val_idx = np.concatenate((np.arange(44,85,10), np.arange(45,86,10),
                          np.arange(130,171,10), np.arange(131,172,10), 
                          np.arange(372,413,10), np.arange(373,414,10)))
'''

# Selects first three years of every decade from 2050 onward as validation
'''
val_idx = np.concatenate((np.arange(44,85,10), np.arange(45,86,10), np.arange(46,77,10),
                          np.arange(130,171,10), np.arange(131,172,10), np.arange(132,163,10),
                          np.arange(372,413,10), np.arange(373,414,10), np.arange(374,405,10)))
'''

# Selects first two years of every decade from 2020 onward as validation
'''
val_idx = np.concatenate((np.arange(14,85,10), np.arange(15,86,10),
                          np.arange(100,171,10), np.arange(101,172,10), 
                          np.arange(342,413,10), np.arange(343,414,10)))
'''

# Selects first two years of every decade from 1850 onward as validation
val_idx = np.concatenate((np.arange(4,85,10), np.arange(5,86,10),
                          np.arange(90,171,10), np.arange(91,172,10), 
                          np.arange(332,413,10), np.arange(333,414,10),
                          np.arange(414,565,10), np.arange(415,566,10),
                          np.arange(570,721,10), np.arange(571,722,10)))

# Selects continuous chunk of data from 2000s as validation
#val_idx = np.concatenate((np.arange(0,45), np.arange(86,131), np.arange(328,373)))

X_train_dict = {}
Y_train_dict = {}
X_val_dict = {}
Y_val_dict = {}

# Create training data
for var in vars_to_predict:
  X, Y, meanstd_inputs = create_training_data(simus, var_to_predict=var)
    
  X_val = np.take(X, val_idx, axis=0)
  X_train = np.delete(X, val_idx, axis=0)
  Y_val = np.take(Y, val_idx, axis=0)
  Y_train = np.delete(Y, val_idx, axis=0)
    
  X_train_dict[var] = X_train
  X_val_dict[var] = X_val
  Y_train_dict[var] = Y_train
  Y_val_dict[var] = Y_val

# Open, reformat, and normalize test data
X_test = xr.open_mfdataset([data_path + 'inputs_historical.nc',
                            data_path + 'inputs_ssp245.nc']).compute()
Y_test = create_predictdand_data(['ssp245'])

for input_var in ['CO2', 'CH4', 'SO2', 'BC']: 
  var_dims = X_test[input_var].dims
  X_test = X_test.assign({input_var: (var_dims, normalize(X_test[input_var].data, input_var, meanstd_inputs))}) 
    
X_test_np = input_for_training(X_test, skip_historical=False, len_historical=165) 

X_train_dict['tas'].shape, Y_train_dict['tas'].shape, X_val_dict['tas'].shape, Y_val_dict['tas'].shape

((608, 10, 96, 144, 4),
 (608, 1, 96, 144),
 (118, 10, 96, 144, 4),
 (118, 1, 96, 144))

In [ ]:
param_lims = {1000000: '1M', 10000000: '10M'}
layer_range = [2,6,10]
num_models = 10

'''
var = 'tas'
var_idx = vars_to_predict.index(var)

X_train = X_train_dict[var]
Y_train = Y_train_dict[var]
X_val = X_val_dict[var]
Y_val = Y_val_dict[var]
'''
vars_to_predict = ['tas', 'diurnal_temperature_range', 'pr', 'pr90']

rmse_diffs = []
rmse_diffs_orig = []

rmse_rand_model_1M_raw = []
rmse_rand_model_10M_raw = []
rmse_frozen_model_1M_raw = []
rmse_frozen_model_10M_raw = []

In [ ]:
cnn_lstm_1M_total = np.load('./drive/My Drive/Colab Notebooks/USC Random NN/experimental_results/cnn_lstm/new_metrics_experiment/1M/rmse_data_total.npy')
cnn_lstm_10M_total = np.load('./drive/My Drive/Colab Notebooks/USC Random NN/experimental_results/cnn_lstm/new_metrics_experiment/10M/rmse_data_total.npy')
cnn_lstm_1M_models_loc = './drive/My Drive/Colab Notebooks/USC Random NN/experimental_results/cnn_lstm/new_metrics_experiment/1M/models/'
cnn_lstm_10M_models_loc = './drive/My Drive/Colab Notebooks/USC Random NN/experimental_results/cnn_lstm/new_metrics_experiment/10M/models/'

cnn_lstm_rand_dense_1M_total = np.load('./drive/My Drive/Colab Notebooks/USC Random NN/experimental_results/cnn_lstm_rand_dense/new_metrics_experiment/1M/rmse_data_total.npy')
cnn_lstm_rand_dense_10M_total = np.load('./drive/My Drive/Colab Notebooks/USC Random NN/experimental_results/cnn_lstm_rand_dense/new_metrics_experiment/10M/rmse_data_total.npy')
cnn_lstm_rand_dense_1M_models_loc = './drive/My Drive/Colab Notebooks/USC Random NN/experimental_results/cnn_lstm_rand_dense/new_metrics_experiment/1M/models/'
cnn_lstm_rand_dense_10M_models_loc = './drive/My Drive/Colab Notebooks/USC Random NN/experimental_results/cnn_lstm_rand_dense/new_metrics_experiment/10M/models/'

In [ ]:
for param_lim in param_lims.keys():

  if param_lim == 1000000:
    dense_rmse_data = cnn_lstm_1M_total
    dense_models_loc = cnn_lstm_1M_models_loc
    rand_dense_rmse_data = cnn_lstm_rand_dense_1M_total
    rand_dense_models_loc = cnn_lstm_rand_dense_1M_models_loc
  else:
    dense_rmse_data = cnn_lstm_10M_total
    dense_models_loc = cnn_lstm_10M_models_loc
    rand_dense_rmse_data = cnn_lstm_rand_dense_10M_total
    rand_dense_models_loc = cnn_lstm_rand_dense_10M_models_loc

  for n_layers in layer_range:

      idx = n_layers - 2

      for i in range(num_models):

        for var in vars_to_predict:
          
          var_idx = vars_to_predict.index(var)

          keras.backend.clear_session()
          stand_model = None
          rand_model = None
          new_model = None

          # Get train/val data
          X_train = X_train_dict[var]
          Y_train = Y_train_dict[var]
          X_val = X_val_dict[var]
          Y_val = Y_val_dict[var]

          # Get best standard model
          best_stand_model_idx = np.argmin(dense_rmse_data[idx,var_idx])
          config = np.load(f'{dense_models_loc}{n_layers}_layer_model_{best_stand_model_idx}.npy', allow_pickle=True)
          stand_model = Sequential.from_config(config.item())
          #stand_model.summary()

          # Train best standard model
          stand_model.compile(optimizer="adam", loss="mse", metrics=["mse"])
          hist = stand_model.fit(X_train,
                                 Y_train,
                                 batch_size=16,
                                 epochs=100,
                                 verbose=0,
                                 validation_data=(X_val,Y_val),
                                 callbacks=EarlyStopping(patience=10, restore_best_weights=True),
                                )
          
          # Get best random model
          best_rand_model_idx = np.argmin(rand_dense_rmse_data[idx,var_idx])
          config = np.load(f'{rand_dense_models_loc}{n_layers}_layer_model_{best_rand_model_idx}.npy', allow_pickle=True)
          custom_objects = {"ApplyPosWeight": ApplyPosWeight}
          with tf.keras.utils.custom_object_scope(custom_objects):
            rand_model = Model.from_config(config.item())
          #rand_model.summary()

          # Get the random layers from the best random model
          rand_layers = [layer.__class__.__name__ for layer in rand_model.layers]
          rand_start_idx = rand_layers.index('TFOpLambda')
          rand_block = Model(rand_model.layers[rand_start_idx-1].output, rand_model.layers[-1].output)
          #rand_block.summary()

          # Get non-random layers from standard model, add random layers on top to create new model
          stand_layers = [layer.__class__.__name__ for layer in stand_model.layers]
          stand_end_idx = stand_layers.index('Dense')-1
          new_output = rand_block(stand_model.layers[stand_end_idx].output)
          new_model = Model(stand_model.input, new_output)
          #new_model.summary()

          # Freeze non-random layers of new model
          for j in range(rand_start_idx):
            new_model.layers[j].trainable = False

          # Train and evaluate best random model
          rand_model.compile(optimizer="adam", loss="mse", metrics=["mse"])
          hist = rand_model.fit(X_train,
                                Y_train,
                                batch_size=16,
                                epochs=100,
                                verbose=0,
                                validation_data=(X_val,Y_val),
                                callbacks=EarlyStopping(patience=10, restore_best_weights=True),
                               )
          m_pred = rand_model.predict(X_test_np)
          m_pred = m_pred.reshape(m_pred.shape[0], m_pred.shape[2], m_pred.shape[3])
          m_pred = xr.DataArray(m_pred, dims=['time','lat','lon'], coords=[X_test.time.data[slider-1:], X_test.latitude.data, X_test.longitude.data])
          m_pred = m_pred.transpose('lat','lon','time').sel(time=slice(2015,2101)).to_dataset(name=var)
          var_truth = Y_test[var]
          m_var_pred = m_pred.transpose('time','lat','lon')[var]
          rmse_spatial = get_rmse_spatial(var_truth[65:], m_var_pred[65:])
          rmse_global = get_rmse_global(var_truth[65:], m_var_pred[65:])
          rmse_total = rmse_spatial + 5*rmse_global
          if param_lim == 1000000:
            rmse_rand_model_1M_raw.append(rmse_total)
          else:
            rmse_rand_model_10M_raw.append(rmse_total)

          # Train and evaluate new model (frozen weights in non-random layers)
          new_model.compile(optimizer="adam", loss="mse", metrics=["mse"])
          hist = new_model.fit(X_train,
                               Y_train,
                               batch_size=16,
                               epochs=100,
                               verbose=0,
                               validation_data=(X_val,Y_val),
                               callbacks=EarlyStopping(patience=10, restore_best_weights=True),
                              )
          m_pred = new_model.predict(X_test_np)
          m_pred = m_pred.reshape(m_pred.shape[0], m_pred.shape[2], m_pred.shape[3])
          m_pred = xr.DataArray(m_pred, dims=['time','lat','lon'], coords=[X_test.time.data[slider-1:], X_test.latitude.data, X_test.longitude.data])
          m_pred = m_pred.transpose('lat','lon','time').sel(time=slice(2015,2101)).to_dataset(name=var)
          var_truth = Y_test[var]
          m_var_pred = m_pred.transpose('time','lat','lon')[var]
          rmse_spatial = get_rmse_spatial(var_truth[65:], m_var_pred[65:])
          rmse_global = get_rmse_global(var_truth[65:], m_var_pred[65:])
          rmse_total = rmse_spatial + 5*rmse_global
          if param_lim == 1000000:
            rmse_frozen_model_1M_raw.append(rmse_total)
          else:
            rmse_frozen_model_10M_raw.append(rmse_total)

In [ ]:
# Some data reshaping for easier analysis
rmse_rand_model_1M_raw = np.array(rmse_rand_model_1M_raw)
rmse_rand_model_1M_raw = np.reshape(rmse_rand_model_1M_raw, (len(layer_range),len(vars_to_predict)*num_models))
rmse_rand_model_1M = np.zeros((len(layer_range),len(vars_to_predict),num_models))
for i in range(rmse_rand_model_1M_raw.shape[0]):
  rmse_rand_model_1M[i] = np.reshape(rmse_rand_model_1M_raw[i], (len(vars_to_predict),num_models), order='F')

rmse_rand_model_10M_raw = np.array(rmse_rand_model_10M_raw)
rmse_rand_model_10M_raw = np.reshape(rmse_rand_model_10M_raw, (len(layer_range),len(vars_to_predict)*num_models))
rmse_rand_model_10M = np.zeros((len(layer_range),len(vars_to_predict),num_models))
for i in range(rmse_rand_model_10M_raw.shape[0]):
  rmse_rand_model_10M[i] = np.reshape(rmse_rand_model_10M_raw[i], (len(vars_to_predict),num_models), order='F')

rmse_frozen_model_1M_raw = np.array(rmse_frozen_model_1M_raw)
rmse_frozen_model_1M_raw = np.reshape(rmse_frozen_model_1M_raw, (len(layer_range),len(vars_to_predict)*num_models))
rmse_frozen_model_1M = np.zeros((len(layer_range),len(vars_to_predict),num_models))
for i in range(rmse_frozen_model_1M_raw.shape[0]):
  rmse_frozen_model_1M[i] = np.reshape(rmse_frozen_model_1M_raw[i], (len(vars_to_predict),num_models), order='F')

rmse_frozen_model_10M_raw = np.array(rmse_frozen_model_10M_raw)
rmse_frozen_model_10M_raw = np.reshape(rmse_frozen_model_10M_raw, (len(layer_range),len(vars_to_predict)*num_models))
rmse_frozen_model_10M = np.zeros((len(layer_range),len(vars_to_predict),num_models))
for i in range(rmse_frozen_model_10M_raw.shape[0]):
  rmse_frozen_model_10M[i] = np.reshape(rmse_frozen_model_10M_raw[i], (len(vars_to_predict),num_models), order='F')

In [ ]:
np.save(results_path+f'rmse_rand_model_1M', rmse_rand_model_1M)
np.save(results_path+f'rmse_rand_model_10M', rmse_rand_model_10M)
np.save(results_path+f'rmse_frozen_model_1M', rmse_frozen_model_1M)
np.save(results_path+f'rmse_frozen_model_10M', rmse_frozen_model_10M)